In [ ]:
from smarty.forcefield import *
import parmed
import openeye.oechem as oechem
from simtk.openmm.app import PDBFile

In [ ]:
pdbfile = PDBFile('cyclohexane_ethanol_0.4_0.6.pdb')
ff = ForceField('../../smarty/data/forcefield/Frosst_AlkEtOH.ffxml')

mols = []
flavor = oechem.OEIFlavor_Generic_Default | oechem.OEIFlavor_MOL2_Default | oechem.OEIFlavor_MOL2_Forcefield
for molname in ['cyclohexane', 'ethanol']:
    ifs = oechem.oemolistream(molname+'.mol2')
    ifs.SetFlavor( oechem.OEFormat_MOL2, flavor)
    mol = oechem.OEGraphMol()
    while oechem.OEReadMolecule(ifs, mol):
        oechem.OETriposAtomNames(mol)
        mols.append(oechem.OEGraphMol(mol))

system = ff.createSystem( pdbfile.topology, mols)
#Swap in positions
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdbfile.positions)

# Create parmed Structure
structure = parmed.openmm.topsystem.load_topology( pdbfile.topology, system, pdbfile.positions)
# Write AMBER parameter/crd
#parmed.amber.AmberParm( system, structure)
#help(structure.save)
#Save prmtop
structure.save('system.prmtop', overwrite=True)
# Save crd
structure.save('system.crd', format='rst7', overwrite=True)



In [ ]:
# Evaluate energy of OpenMM system and AMBER system to check that they are still the same
from smarty.forcefield_utils import *
parm = parmed.load_file('system.prmtop', 'system.crd')
ambersys = parm.createSystem(nonbondedMethod= app.NoCutoff, constraints = None, implicitSolvent = None)
groups0, groups1, energy0, energy1 = compare_system_energies( pdbfile.topology, pdbfile.topology, ambersys, system, pdbfile.positions)

# Load as in create_system_from_amber; this fails. Apparently some sort of OpenMM issue.
#prmtop = app.AmberPrmtopFile('system.prmtop')
#system =prmtop.createSystem(nonbondedMethod = app.NoCutoff, constraints = None, implicitSolvent = None)


In [ ]:
# Dump OpenMM systems out

# Do smarty system
from simtk.openmm import XmlSerializer
serialized_system_smirff = XmlSerializer.serialize(system)
outfile = open('smarty_system.xml','w')
outfile.write(serialized_system_smirff)
outfile.close()
# Do amber system
serialized_system_amber = XmlSerializer.serialize(ambersys)
outfile = open('amber_system.xml','w')
outfile.write(serialized_system_amber)
outfile.close()

In [14]:
# deserialize systems and confirm energies still different without using external tools
smirff_system = XmlSerializer.deserialize(serialized_system_smirff)
amber_system = XmlSerializer.deserialize(serialized_system_amber)
pdbfile = PDBFile('cyclohexane_ethanol_0.4_0.6.pdb')

# Do an energy comparison
timestep = 1.0 * unit.femtoseconds
integrator0 = simtk.openmm.VerletIntegrator( timestep )
integrator1 = simtk.openmm.VerletIntegrator( timestep )
platform = simtk.openmm.Platform.getPlatformByName("Reference")
positions = pdbfile.positions
simulation0 = app.Simulation( pdbfile.topology, smirff_system, integrator0, platform = platform )
simulation0.context.setPositions(positions)
simulation1 = app.Simulation( parm.topology, amber_system, integrator1, platform = platform )
simulation1.context.setPositions(positions)
syscheck = system_checker.SystemChecker( simulation0, simulation1 )
groups0, groups1 = syscheck.check_energy_groups(skip_assert = False)
energy0, energy1 = syscheck.check_energies(skip_assert = False)

In [15]:
# Save mol2 file for swails
structure.save('swails.mol2')

IOError: swails.mol2 exists; not overwriting

In [16]:
# Debugging as per Swails
import parmed as pmd
omm = pmd.openmm.load_topology(pmd.load_file('swails.mol2', structure=True).topology, pmd.load_file('smarty_system.xml'))
omm.save('test.parm7', overwrite=True)
omm.coordinates = pmd.load_file('swails.mol2', structure=True).coordinates
omm.save('test.rst7', overwrite=True)
parm = pmd.load_file('test.parm7', 'test.rst7')
omm.coordinates = parm.coordinates # Make sure coordinates are identical
pmd.openmm.energy_decomposition_system(omm, pmd.load_file('smarty_system.xml'))


[('HarmonicBondForce', 42.56436386960996),
 ('HarmonicAngleForce', 109.54924584764584),
 ('PeriodicTorsionForce', 381.4186716756578),
 ('NonbondedForce', 12147.284380939787),
 ('CMMotionRemover', 0.0)]

In [17]:
pmd.openmm.energy_decomposition_system(parm, parm.createSystem())

[('HarmonicBondForce', 42.56436386960996),
 ('HarmonicAngleForce', 109.54924584764584),
 ('PeriodicTorsionForce', 381.41867167565795),
 ('NonbondedForce', 12147.28437186874),
 ('CMMotionRemover', 0.0)]

In [18]:
omm2 = pmd.openmm.load_topology(pmd.load_file('swails.mol2', structure=True).topology, pmd.load_file('amber_system.xml'))
omm2.coordinates = parm.coordinates
pmd.openmm.energy_decomposition_system(omm2, pmd.load_file('amber_system.xml'))

[('HarmonicBondForce', 42.56436386960996),
 ('HarmonicAngleForce', 109.54924584764584),
 ('PeriodicTorsionForce', 381.41867167565795),
 ('NonbondedForce', 12147.28437186874),
 ('CMMotionRemover', 0.0)]